# 📚 Importing Libraries

In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

# ⚙️ Configuration Class

In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10


# 📊 Loading and Processing Data

we will load our training and test datasets, and apply some preprocessing. This includes:


1. **Loading Data**: Read the CSV files into pandas DataFrames.
2. **Subsampling**: If the test dataset has less than 10 rows, subsample the training dataset to 10,000 rows for quicker processing.
3. **Processing Strings**: Clean and process the string columns (`prompt`, `response_a`, `response_b`) by removing unwanted characters.
4. **Shape and Missing Values**: Print the shape of the datasets and count missing values to understand the data structure and quality.

In [ ]:
train = pd.read_csv(config.train_path)
test = pd.read_csv(config.test_path)
sample_submission = pd.read_csv(config.sample_submission_path)

if test.shape[0] < 10:
    train = train.iloc[:10000]
    
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

train.head()

# 🛠️ Preprocessing Class Definition

This class, `Preprocessor`, contains several methods to process and feature-engineer the text data. Here’s a breakdown of its functionalities:

#### Cosine Similarity
- **Formula**: 
  $$\text{cosine_similarity} = \frac{A \cdot B}{\|A\| \|B\|}$$

- **Description**: Cosine similarity measures the cosine of the angle between two vectors, providing a metric of how similar the texts are.

#### Jaccard Similarity
- **Formula**:

  $$\text{Jaccard_similarity} = \frac{|A \cap B|}{|A \cup B|}$$

- **Description**: Jaccard similarity measures the similarity between two sets by comparing their intersection and union.

#### Count Quotes
- **Description**: This method identifies and counts both single and double quoted texts in a string, giving an idea of how many quotations are present.

#### Tokenize
- **Description**: This method splits the text into individual words (tokens), which can be used for further analysis like generating n-grams or calculating overlaps.

#### Generate N-grams
- **Description**: N-grams are contiguous sequences of 'n' items from a given text. This method helps in analyzing the text at different levels of granularity (unigrams, bigrams, trigrams, etc.).

#### Count N-gram Overlaps
- **Description**: This method calculates how many n-grams are common between two texts, helping to measure their similarity.

#### Run
- **Description**: This method processes the entire dataset, generating new features based on the above calculations, which can be used for training machine learning models.


$\frac{n!}{k!(n-k)!} = \binom{n}{k}$


In [ ]:
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer().fit_transform([text1, text2])
            vectors = vectorizer.toarray()
            cos_sim = cosine_similarity(vectors)
            return cos_sim[0][1]
        except:
            return np.nan

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())
        set2 = set(text2.split())
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        return len(intersection) / len(union)
    
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"
        double_quote_pattern = r'"(.*?)"'
        single_quotes = re.findall(single_quote_pattern, text)
        double_quotes = re.findall(double_quote_pattern, text)
        total_quotes = len(single_quotes) + len(double_quotes)
        return len(single_quotes) + len(double_quotes)

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)
        return list(ngrams(tokens, n))

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)
            ngrams2 = self.generate_ngrams(text2, n)
            counter1 = Counter(ngrams1)
            counter2 = Counter(ngrams2)
            overlap = counter1 & counter2
            overlap_count = sum(overlap.values())
            return overlap_count
        except:
            return 0
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))
        
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]
        
        data["respa_prompt_overlap_unigram_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        data["respb_prompt_overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))
        
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        return data

In [ ]:
%%time

preprocessor = Preprocessor()
train = preprocessor.run(train)
test = preprocessor.run(test)
train.head()

### Data Preparation


In [ ]:
drop_cols = ["id", "response_a", "response_b", "prompt"]
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]
target = "target"

train[target] = np.nan
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx
train[target] = train[target].astype("int32")
    
train.head()

In [ ]:
X = train.drop(columns=target_cols+drop_cols+[target]+["model_a", "model_b"], axis=1)
y = train[target]
X_test = test.drop(columns=drop_cols, axis=1)

X = X.replace([-np.inf, np.inf], np.nan)
X_test = X_test.replace([-np.inf, np.inf], np.nan)

In [ ]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

In [ ]:
# Feature Selection
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)
X_test_new = selector.transform(X_test)

# 🧩 Model Training and Evaluation

### Model Definitions

Defines several machine learning models including Random Forest, Gradient Boosting, SVM, XGBoost, CatBoost, and a Voting Classifier.

### Feature Selection

Uses SelectKBest to select 25 best features based on ANOVA F-value between feature and target.

### Cross-validation

Uses Stratified K-Fold cross-validation for model evaluation.

### Training and Evaluation

Iterates through each model, trains it using the training data, evaluates using cross-validation, and calculates the mean CV Log Loss.

### Feature Importances

Calculates and stores feature importances for applicable models (Random Forest, Gradient Boosting, XGBoost, CatBoost).

### Best Model Identification

Identifies the best performing model based on the lowest CV Log Loss.

### Results Display

Displays the results in a DataFrame showing the CV Log Loss for each model and, if applicable, feature importances.


In [ ]:

# Define models and their configurations
models = {
    'random_forest': {
        'model': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=config.seed
        ),
        'params': {}
    },
    'gradient_boosting': {
        'model': GradientBoostingClassifier(
            n_estimators=300,
            learning_rate=0.1,
            max_depth=5,
            subsample=0.8,
            random_state=config.seed
        ),
        'params': {}
    },
    'svm': {
        'model': SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=config.seed
        ),
        'params': {}
    },
    'xgboost': {
        'model': xgb.XGBClassifier(
            objective='multi:softprob',
            num_class=3,
            eval_metric='mlogloss',
            subsample=0.8,
            n_estimators=650,
            learning_rate=0.045,
            max_depth=5,
            random_state=config.seed,
#             tree_method='gpu_hist'  # GPU acceleration if available
        ),
        'params': {}
    },
    'catboost': {
        'model': cb.CatBoostClassifier(
            loss_function='MultiClass',
            iterations=650,
            learning_rate=0.045,
            depth=5,
            random_seed=config.seed,
#             task_type="GPU",  # Use GPU if available
            verbose=75
        ),
        'params': {}
    },
    'voting': {
        'model': VotingClassifier(
            estimators=[
                ('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)),
                ('svc', SVC(probability=True)),
                ('knn', KNeighborsClassifier(n_neighbors=5))
            ],
            voting='soft'
        ),
        'params': {}
    }
}

# Select features using SelectKBest
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)
X_test_new = selector.transform(X_test)

# Cross-validation setup
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

# Dataframe to store results
results = []

# Iterate over models
for model_name, model_data in models.items():
    model = model_data['model']
    print(f"Training model: {model_name}")

    test_preds = np.zeros(shape=(X_test_new.shape[0], y.nunique()))
    cv_scores = []

    for idx, (train_idx, val_idx) in enumerate(cv.split(X_new, y)):
        X_train, y_train = X_new[train_idx], y[train_idx]
        X_val, y_val = X_new[val_idx], y[val_idx]

        if model_name == 'voting':
            model.fit(X_train, y_train)
        elif model_name == 'catboost':
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                early_stopping_rounds=75,
                verbose=75
            )
        else:
            model.fit(
                X_train,
                y_train
            )

        if model_name != 'voting':
            val_preds = model.predict_proba(X_val)
            val_log_loss = log_loss(y_val, val_preds, eps="auto")
            cv_scores.append(val_log_loss)

            test_preds += model.predict_proba(X_test_new) / cv.get_n_splits()

    if model_name != 'voting':
        mean_cv_log_loss = np.mean(cv_scores)
        results.append({'Model': model_name, 'CV_Log_Loss': mean_cv_log_loss})
        print(f"Mean CV Log Loss: {mean_cv_log_loss:.5f}")

# Store feature importances if applicable
if model_name in ['random_forest', 'gradient_boosting', 'xgboost', 'catboost']:
    features = X.columns[selector.get_support()].tolist()
    feat_imp_df = pd.DataFrame({"feature": features})
    feat_imp_df[f"{model_name}_avg_importance"] = 0

    for idx, (_, val_idx) in enumerate(cv.split(X_new, y)):
        X_val, _ = X_new[val_idx], y[val_idx]
        feat_imp_df[f"{model_name}_avg_importance"] += model.feature_importances_ / cv.get_n_splits()

    results_df = pd.DataFrame(results)
    results_df = pd.concat([results_df, feat_imp_df], axis=1)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best model
best_model = results_df.loc[results_df['CV_Log_Loss'].idxmin()]
print(f"\nBest Model:\n{best_model}")

# Display results DataFrame
print("\nResults DataFrame:")
print(results_df)


In [ ]:
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]
sample_submission.head()

In [ ]:
sample_submission.to_csv("submission.csv", index=False)